In [30]:
import torch
import seqm
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure

torch.set_default_dtype(torch.float64)
# if torch.cuda.is_available():
#     device = torch.device('cuda')
# else:
device = torch.device('cpu')
dtype = torch.float64

%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from seqm.seqm_functions.fock import fock
from seqm.seqm_functions.pack import unpack
import seqm.seqm_functions.pack as pack
from icecream import ic as ic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [968]:
ic.disable()

In [535]:
%%time

### create molecule object:
species = torch.as_tensor([[8,1,1]], # zero-padding for batching
                          dtype=torch.int64, device=device)

coordinates = torch.tensor([
                              [
                               [-0.368758544,      0.000000000,     -1.211413881],
                               [-0.368758544,      0.759337000,     -0.615370881],
                               [-0.368758544,     -0.759337000,     -0.615370881],
                              ]
                            ], device=device)

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

seqm_parameters = {
                   'method' : 'PM3',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True,
                   'excited' : True,
                   }

m = seqm.Molecule.Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on m:
esdriver(m)

species tensor([[8, 1, 1]])
species.shape torch.Size([1, 3])
species tensor([[8, 1, 1]])
species.shape torch.Size([1, 3])
tore Parameter containing:
tensor([0., 1., 0., 1., 2., 3., 4., 5., 6., 7., 0., 1., 2., 3., 4., 5., 6., 7., 0.])
qn Parameter containing:
tensor([0., 1., 0., 2., 2., 2., 2., 2., 2., 2., 0., 3., 3., 3., 3., 3., 3., 3., 0.])
nmol 1
molsize 3
P0 torch.Size([1, 12, 12])
tensor([[[1.50000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000],
         [0.00000, 1.50000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000],
         [0.00000, 0.00000, 1.50000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000],
         [0.00000, 0.00000, 0.00000, 1.50000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000],
         [0.00000, 0.00000, 0.00000, 0.00000, 1.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000],
         [0.00000

In [536]:
m.parameters

{'U_ss': tensor([-86.99300, -13.07332, -13.07332]),
 'U_pp': tensor([-71.87958,   0.00000,   0.00000]),
 'zeta_s': tensor([3.79654, 0.96781, 0.96781]),
 'zeta_p': tensor([2.38940, 0.00000, 0.00000]),
 'beta_s': tensor([-45.20265,  -5.62651,  -5.62651]),
 'beta_p': tensor([-24.75251,   0.00000,   0.00000]),
 'g_ss': tensor([15.75576, 14.79421, 14.79421]),
 'g_sp': tensor([10.62116,  0.00000,  0.00000]),
 'g_pp': tensor([13.65402,  0.00000,  0.00000]),
 'g_p2': tensor([12.40610,  0.00000,  0.00000]),
 'h_sp': tensor([0.59388, 0.00000, 0.00000]),
 'alpha': tensor([3.21710, 3.35639, 3.35639]),
 'Gaussian1_K': tensor([-1.13113,  1.12875,  1.12875]),
 'Gaussian2_K': tensor([ 1.13789, -1.06033, -1.06033]),
 'Gaussian1_L': tensor([6.00248, 5.09628, 5.09628]),
 'Gaussian2_L': tensor([5.95051, 6.00379, 6.00379]),
 'Gaussian1_M': tensor([1.60731, 1.53747, 1.53747]),
 'Gaussian2_M': tensor([1.59840, 1.57019, 1.57019])}

In [625]:
def decompose_to_sym_antisym(A):
        

    A_sym = 0.5 * (A + A.T)
    A_antisym = 0.5 * (A - A.T)
    
    return A_sym, A_antisym

In [795]:
def ao2mo(M_ao, C, full=False):
    """
    transform matrix from AO to MO basis

    Parameters
    ----------
    M_AO : torch tensor # TODO add size
        matrix in AO basis
    C : torch tensor # TODO add size
        matrix of MO coefficients # TODO row or columns, structure?
        
    Returns
    -------
    M_MO : torch tensor # TODO add size
        matrix in MO basis
    """    
    if full == True:
        M_mo = C.T @ M_ao @ C
        return M_mo
        
    else:
         # COPY of subroutine site2mo from Lioville
         
        G_ao = M_ao # TODO rename
        
        # eta1 = eta1.view(-1, m.nvirt[0]) # 1d -> 2d
        # print(eta1.shape)
        # print('eta1', eta1)
        # print('==============')
        
        eta_mo = torch.zeros((N_rpa))
       # eta_mo = torch.zeros((m.norb, m.norb), device=device)

        dgemm1 = G_ao.T @ m.C_mo[0]

        # print('dgemm1.shape', dgemm1.shape)
        # print(dgemm1)
        
        dgemm2 =  m.C_mo[0][:, m.nocc:m.norb].T @ dgemm1[:,:m.nocc]
        

        dgemm2 = dgemm2.T.flatten()
        eta_mo[:dgemm2.size(0)] = dgemm2 
        # print('eta_mo', eta_mo.shape)
        # print(eta_mo)
        
        dgemm3 =  dgemm1[:, m.nocc:].T @ m.C_mo[0][:, :m.nocc]
        
        # print('dgemm3.T.shape', dgemm3.T.shape)
        # print(dgemm3.T)
        
        eta_mo[N_cis:] = dgemm3.T.flatten() 
        # print('eta_mo', eta_mo.shape)
        # print(eta_mo)

        M_mo = eta_mo
    
    return M_mo

In [796]:
davidson_vec()

molecules Molecule(
  (const): Constants()
  (parser): Parser()
)


AttributeError: module 'torch' has no attribute 'copy'

In [797]:
def mo2ao(M_mo, molecule, full=False):
    """
    transform matrix from AO to MO basis

    Parameters
    ----------
    M_AO : torch tensor # TODO add size
        matrix in AO basis
    C : torch tensor # TODO add size
        matrix of MO coefficients # TODO row or columns, structure?
        
    Returns
    -------
    M_MO : torch tensor # TODO add size
        matrix in MO basis
    """    
    
    if full == True:
        M_ao = C.T @ M_mo @ C #! does not currently work
        
        return M_ao
    else:
        
        eta = M_mo # TODO rename
        
        eta1 = eta[:N_cis]
        eta1 = eta1.view(-1, m.nvirt[0]) # 1d -> 2d
        # print(eta1.shape)
        # print('eta1', eta1)
        # print('==============')
        
        eta_mo = torch.zeros((m.norb, m.norb), device=device)

        dgemm1 = eta1 @ m.C_mo[0][:, m.nocc:m.norb].T # operations on |X| ?

        # print('dgemm1.shape', dgemm1.shape)
        # print(dgemm1)
        
        eta_mo[:m.nocc] = dgemm1
        # print('eta_mo', eta_mo.shape)
        # print(eta_mo)
        
        
        eta2 = eta[N_cis:]                            # operations on |Y| ?
        eta2 = eta2.view(-1, m.nvirt[0]) # 1d -> 2d
    
        dgemm2 = eta2.T @ m.C_mo[0][:, :m.nocc].T
        
        # print('dgemm2.shape', dgemm2.shape)
        # print(dgemm2)
        
        eta_mo[m.nocc:] = dgemm2
        # print('eta_mo', eta_mo.shape)
        # print(eta_mo)
        
        dgemm3 = m.C_mo[0] @ eta_mo
        eta_ao = dgemm3 
    
    
        return eta_ao

In [798]:
def build_G_sym(M_ao,
                gss, gsp, gpp, gp2, hsp,
                mask, maskd, idxi, idxj, nmol, molsize,
                w):
    
    
      F = torch.zeros((nmol*molsize**2,4,4), device=device) # 0 Fock matrix to fill
      # # TODO: feed params programmatically
      P0 = unpack(M_ao, 1, 2, 12) # 
      P0 = torch.unsqueeze(P0, 0) # add dimension
      
      # print('P0.shape', P0.shape)
      # print('P0\n', P0)
      #---------------fill diagonal 1c-2e -------------------
      P = P0.reshape((nmol,molsize,4,molsize,4)) \
          .transpose(2,3).reshape(nmol*molsize*molsize,4,4)
          
      # print('P.shape', P.shape)
      # print('P\n', P)
      
      Pptot = P[...,1,1]+P[...,2,2]+P[...,3,3]
      ## http://openmopac.net/manual/1c2e.html
    #  (s,s)
      TMP = torch.zeros_like(F)
      TMP[maskd,0,0] = 0.5*P[maskd,0,0]*gss + Pptot[maskd]*(gsp-0.5*hsp)
      for i in range(1,4):
          #(p,p)
          TMP[maskd,i,i] = P[maskd,0,0]*(gsp-0.5*hsp) + 0.5*P[maskd,i,i]*gpp \
                          + (Pptot[maskd] - P[maskd,i,i]) * (1.25*gp2-0.25*gpp)
          #(s,p) = (p,s) upper triangle
          TMP[maskd,0,i] = P[maskd,0,i]*(1.5*hsp - 0.5*gsp)
      #(p,p*)
      for i,j in [(1,2),(1,3),(2,3)]:
          TMP[maskd,i,j] = P[maskd,i,j]* (0.75*gpp - 1.25*gp2)

      F.add_(TMP)
      
           
      #-----------------fill 2c-2e integrals----------------
      weight = torch.tensor([1.0,
                        2.0, 1.0,
                        2.0, 2.0, 1.0,
                        2.0, 2.0, 2.0, 1.0],dtype=dtype, device=device).reshape((-1,10))
      
      PA = (P[maskd[idxi]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,10,1))
      PB = (P[maskd[idxj]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,1,10))
      suma = torch.sum(PA*w,dim=1)
      sumb = torch.sum(PB*w,dim=2)
      sumA = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      sumB = torch.zeros_like(sumA)
      
      sumA[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = suma
      sumB[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = sumb
      F.index_add_(0,maskd[idxi],sumB)
      #\sum_A
      F.index_add_(0,maskd[idxj],sumA)
      
      sum = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      # (ss ), (px s), (px px), (py s), (py px), (py py), (pz s), (pz px), (pz py), (pz pz)
      #   0,     1         2       3       4         5       6      7         8        9

      ind = torch.tensor([[0,1,3,6],
                          [1,2,4,7],
                          [3,4,5,8],
                          [6,7,8,9]],dtype=torch.int64, device=device)
      
      Pp = -0.5*P[mask]
      for i in range(4):
        for j in range(4):
            #\sum_{nu \in A} \sum_{sigma \in B} P_{nu, sigma} * (mu nu, lambda, sigma)
            sum[...,i,j] = torch.sum(Pp*w[...,ind[i],:][...,:,ind[j]],dim=(1,2))
      #print('mask', mask)    #! DIFFERS FROM PYSEQM, PROBABLY packing
      F.index_add_(0,mask,sum)

      F0 = F.reshape(nmol,molsize,molsize,4,4) \
             .transpose(2,3) \
             .reshape(nmol, 4*molsize, 4*molsize)
    #
      F0.add_(F0.triu(1).transpose(1,2))     
      
      F0 = 2 * F0 #! BE CAREFUL
      # print('F0.shape', F0.shape)
      # print(F0) 
      
      return F0

In [799]:
def build_G_antisym(eta_ao, eta_ao_asym, G_sym,
                gss, gsp, gpp, gp2, hsp,
                mask, maskd, idxi, idxj, nmol, molsize,
                w, 
                m):

      # TODO; figure how/why constants are defined in fock_skew
      #!
      #! CHECK
      #!
      gss = torch.tensor([15.7558, 14.7942, 14.7942])  # GSSII   7.87788  7.39710   7.39710   1/2 # not used
      gpp = torch.tensor([13.6540,  0.0000,  0.0000])  # GPPII   6.82700   1/2
      gsp = torch.tensor([10.6212,  0.0000,  0.0000])  # GSPII   10.6211    
      gp2 = torch.tensor([12.4061,  0.0000,  0.0000])  # GP2II   15.5076
      hsp = torch.tensor([0.5939, 0.0000, 0.0000])     # HSPII   0.29694   1/2
  
      # see fock_skew in qm_fock in NEXMD
      # create 1d array 
      eta_anti = torch.zeros((m.norb*(m.norb+1)//2), device=device)
      
      l=0
      for i in range(m.norb):
            for j in range(i+1):

                  eta_anti[l] = 0.5 * (eta_ao[i,j] - eta_ao[j,i])
                  l += 1    
      
      eta_anti_2d = torch.zeros((m.norb, m.norb), device='cpu') 

      #restore to 2d form to build G 2c-2e part
      l = 0
      for i in range(0, m.norb): # TODO" vectorize
          for j in range(0,i):
              l += 1
              eta_anti_2d[i,j] += eta_anti[l-1]
              eta_anti_2d[j,i] -= eta_anti[l-1]
          l += 1 
    
    

      pascal2 = [1, 3, 6, 10, 15, 21]  # -1 pascal triangle
      pascal2 = [x -1 for x in pascal2]
      pascal1 = [0, 1, 3, 6, 10, 15] # -1 for python indexing
      pascal1 = [x -1 for x in pascal1]
      orb_loc1 = [0,4,5] # O orbs
      orb_loc2 = [3,4,5]
      
      G_1c2e = torch.zeros((m.norb*(m.norb+1)//2))
    
      for ii in range(molsize): # n_atoms?
      # print('ii', ii)
        if m.Z[ii] == 1:
          pass
        
        else:
          gsp_ii = gsp[ii]
          gpp_ii = gpp[ii]
          gp2_ii = gp2[ii]
          hsp_ii = hsp[ii]
          
          ia = orb_loc1[ii]
          # print('ia', ia)
          ib = orb_loc2[ii]
          
          iplus = ia+1
          ka = pascal2[ia]
          l = ka
          

          
          for j in range(iplus, ib+1):
          # print('j', j)
          # print('ia', ia)
          # print('ib', ib)
          # print('l', l)

            mm = l+ia+1
            l = l+j+1
            
            # print(type(mm)) 
            G_1c2e[mm] = G_1c2e[mm] + 0.5*eta_anti[mm] * (hsp_ii - gsp_ii)
            # print('(hsp - gsp)', (hsp - gsp))
          #  print('G_1c2e[mm]', G_1c2e[mm])
          #  print('===================')
            
          iminus = ib-1
        
          for j in range(iplus, iminus+1):
            icc = j
            # print('icc', icc)
            for l in range(icc, ib):
              # print('l', l)
              mm = pascal1[l+1] + j+1
              # print('mm', mm)
              # print('(0.25*gpp_ii - 0.6*gp2_ii)', (0.25*gpp_ii - 0.6*gp2_ii) )
              G_1c2e[mm] = G_1c2e[mm]+ eta_anti[mm] * (0.25*gpp_ii - 1.25*0.6*gp2_ii) 
            #  print('G_1c2e[mm]', G_1c2e[mm])
            
      G_1c2e = G_1c2e*2 # antisym 1c2e part
      
      # buils antisymmetric part as Vxi_packA, requires G_sym
      
   #   print('G_sym shape', G_sym.shape)
   #   print('G_sym\n', G_sym)
      
     # print('molsize', molsize)
     
      G_sym = G_sym[0] #! works for one mol only?
      
      l = 0
      for i in range(0, m.norb): # TODO vectorize
          for j in range(0,i):
              l += 1

              G_sym[i,j] += G_1c2e[l-1]
              G_sym[j,i] -= G_1c2e[l-1]             
          l += 1 # skip diagonal
      
      # G_anti_1c = torch.zeros((m.norb,m.norb), device=device)
      # l = 0
      # for i in range(0, m.norb): # TODO vectorize
      #     for j in range(0,i):
      #         l += 1

      #         G_anti_1c[i,j] += G_1c2e[l-1]
      #         G_anti_1c[j,i] -= G_1c2e[l-1]             
      #     l += 1 # skip diagonal  
          
          

            
      # print('G_anti_1c shape', G_anti_1c.shape)
      # print('G_anti_1c\n', G_anti_1c)
      
    #  print('eta_ao_asym shape', eta_ao_asym.shape)
    #  print('eta_ao_asym\n', eta_ao_asym)
      
      # G_tmp = 2* G_anti_1c + G_sym
      G_sym = torch.unsqueeze(G_sym, 0) #  add dimension

      # build 2c-2e part of antisymmetric G
      # copied from FOCK
      
      F = torch.zeros((nmol*molsize**2,4,4), device=device) # 0 Fock matrix to fill
      P0 = unpack(eta_ao_asym, 1, 2, 12) # 
      P0 = torch.unsqueeze(P0, 0) # add dimension
      
      P = P0.reshape((nmol,molsize,4,molsize,4)) \
          .transpose(2,3).reshape(nmol*molsize*molsize,4,4)
          
      #P = P[...,1,1]+P[...,2,2]+P[...,3,3] #! MODIFIED
      #-----------------fill 2c-2e integrals----------------
      weight = torch.tensor([1.0,
                        2.0, 1.0,
                        2.0, 2.0, 1.0,
                        2.0, 2.0, 2.0, 1.0],dtype=dtype, device=device).reshape((-1,10))
      
      PA = (P[maskd[idxi]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,10,1))
      PB = (P[maskd[idxj]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,1,10))
      suma = torch.sum(PA*w,dim=1)
      sumb = torch.sum(PB*w,dim=2)
      sumA = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      sumB = torch.zeros_like(sumA)
      
      sumA[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = suma
      sumB[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = sumb
      F.index_add_(0,maskd[idxi],sumB)
      #\sum_A
      F.index_add_(0,maskd[idxj],sumA)
      
      sum = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      # (ss ), (px s), (px px), (py s), (py px), (py py), (pz s), (pz px), (pz py), (pz pz)
      #   0,     1         2       3       4         5       6      7         8        9

      ind = torch.tensor([[0,1,3,6],
                          [1,2,4,7],
                          [3,4,5,8],
                          [6,7,8,9]],dtype=torch.int64, device=device)
      
      Pp = -0.5*P[mask]
      for i in range(4):
        for j in range(4):
            #\sum_{nu \in A} \sum_{sigma \in B} P_{nu, sigma} * (mu nu, lambda, sigma)
            sum[...,i,j] = torch.sum(Pp*w[...,ind[i],:][...,:,ind[j]],dim=(1,2))
     # print('mask', mask)    #! DIFFERS FROM PYSEQM, PROBABLY packing
      F.index_add_(0,mask,sum)

      F0 = F.reshape(nmol,molsize,molsize,4,4) \
             .transpose(2,3) \
             .reshape(nmol, 4*molsize, 4*molsize)
    #
      F0.add_(F0.triu(1).transpose(1,2))     
      
      # F0 is still symmetric, probably symmetrized above
      # here we make it antisymmetric back
      #F0 = 2 * F0 
      F0 = pack.pack(F0, m.nHeavy, m.nHydro)
      
      rows, cols = torch.tril_indices(F0.shape[1], F0.shape[2])
      F0[0][rows, cols] *= -1
      F0[0][torch.eye(F0.shape[1]).bool()] *= -1
      
      

      F0[0].diagonal().fill_(0) #! BE WARNED, THIS iS TAKEN FROM OLD NEXMD, PYSEQM produces non-zero diagonal
      F0 = F0*2
    #  print('G ANTISYM shape', F0.shape)
    #  print('G ANTISYM\n', F0*2)
      G_full = G_sym + F0 # summ of sym 1c2e and 2c2e + antisym 1c2e and 2c2e
      
      return G_full

In [800]:
# form quasidiagonal

N_cis = m.nocc * m.nvirt
N_rpa = 2 * m.nocc * m.nvirt
rrwork = torch.zeros(N_rpa * 4, device=device) # rrwork.shape is 64 for H2O
print('rrwork', rrwork)
print('rrwork.shape', rrwork.shape)

#------------- order quaisidiagonal ----------------

i = 0
for ip in range(m.nocc):
    for ih in range(m.nvirt):
        rrwork[i] = m.e_mo[0][m.nocc + ih] - m.e_mo[0][ip]  # !Lancos vectors(i) ???
        i += 1                                                                      #  TODO: [0] should be replaced by m batch index
                                                                                    # ? why 64 if most are 0?
print(rrwork)

rrwork_sorted, indices = torch.sort(rrwork[:N_cis], descending=False)

#---------- end order quaisidiagonal ----------------

# ! rrdpsort 

# ------- Wilkinson shift ---------------------------
#-------end Wilkinson shift -------------------------




rrwork tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
rrwork.shape torch.Size([64])
tensor([40.61476, 41.67589, 21.34834, 22.40947, 18.73179, 19.79293, 16.35843, 17.41956,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,
         0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,
         0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,
         0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,
         0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000])


In [729]:
torch.set_printoptions(precision=5, linewidth=150, sci_mode=False)

In [1020]:
def davidson_qm(n_V_start, CIS=True):
   
   nd = 10 # dimension of Krylov subspace 
   icount=0
   nd1_old=0
 #  m=0
   n=0
   iloop=0
   itarget = 4
   #print('molecules', m)
   # goto 70 - restart
   
   nd  = 10 # dimension of Krylov subspace; usually 5x of excitations to be computed
   
   vexp1 = torch.zeros((N_cis, nd), device=device)
   row_idx = torch.arange(0, int(N_cis), device=device)
   col_idx = indices
   
   
   vexp1[row_idx, col_idx] = 1.0 
   vexp1[:,itarget:] = 0.0 # return all columns beyond itarget to 0
   
  #  print('vexp1.shape', vexp1.shape)
  #  print('vexp1\n', vexp1)
   
   import torch.nn.functional as F
   M = 4  # Size of the desired block
   pad_size = 6  # Number of rows/columns to pad
   
   # vexp1 is X in RPA?

   V = torch.zeros((N_rpa, n_V_start), device=device)
   V[:vexp1.shape[0], :vexp1.shape[1]] = vexp1[:, :n_V_start]
  # V[:, :n_V_start] = vexp1
   
  #  print('V.shape', V.shape)
  #  print('V\n', V)
   # vexp = vexp1
   # print('vexp.shape', vexp.shape)
   # print('vexp\n', vexp)
   
   # ORTHONORMALIZATION
   # parse params for G routines
   gss = m.parameters['g_ss']
   gsp = m.parameters['g_sp']
   gpp = m.parameters['g_pp']
   gp2 = m.parameters['g_p2']
   hsp = m.parameters['h_sp']
   
   mask  = m.mask
   maskd = m.maskd
   idxi  = m.idxi
   idxj  = m.idxj
   nmol  = m.nmol
   molsize = m.molsize
   w       = m.w
   nHeavy = m.nHeavy
   nHydro = m.nHydro
   
   
   vexp = torch.zeros((N_rpa, N_rpa), device=device) 
   #vexp = torch.zeros((N_rpa, nd), device=device) 
   
   for i in range(n_V_start):  # i=nd1_old+1,nd1  
      
      # Lxi testing
      
      eta = torch.zeros((N_rpa), device=device) 
      eta[:vexp1[:,i].shape[0]] = vexp1[:,i] # eta is stored as |X|; dcopy?
      
      eta_orig = torch.clone(eta)
      # print('eta_orig.shape', eta_orig.shape)
      # print('eta_orig\n', eta_orig)
      
      
      # print('eta.shape', eta.shape)
      # print(eta)
      eta_ao =  mo2ao(eta, m.C_mo[0], full=False)     # mo to ao basis (mo2site)
      # print('eta_ao.shape', eta_ao.shape)
      # print(eta_ao)

      eta_ao_sym, eta_ao_asym = decompose_to_sym_antisym(eta_ao) # decompose to sym and asym
    
      # Vxi - build 2e integrals in AO basis: G(guess density) in F = H_core + G
      # note density is split into sym and anisym matrices
      # sym is processed as padded 3d array in PYSEQM, see fock module 
      # antisym: 2c-2e works with modified PYSEQM routine; should be antisimmterized afterwards
      # antisym: 1c-2e (diagonal) are taken from NEXMD for now - ugly code with loops
      # TODO: vectorize 1c-2e part
      
      #------------------symmetric------------------------------
      G_sym   =  build_G_sym(eta_ao_sym,
                            gss, gsp, gpp, gp2, hsp,
                            mask, maskd, idxi, idxj, nmol, molsize,
                            w)
      # G sym is 1c-2e and 2c-2e of symmetric part of guess density
      
      
      
      # pack 2c-2e part to standard shape
      G_sym = pack.pack(G_sym, nHeavy, nHydro)
      # print('G_sym \n', G_sym)
      
      G_tot = build_G_antisym(eta_ao, eta_ao_asym, G_sym,
                              gss, gsp, gpp, gp2, hsp,
                              mask, maskd, idxi, idxj, nmol, molsize,
                              w, 
                              m) 
      # build_G_antyssym returnsa both sym and antisym
      # TODO: refactor into: 2c-2e antisym, 1c-2e antisym
      # TODO: vectorize 1c-2e antisym, avoid ugly loops
      #! remember about making 2c-2e diagonal 0

      # print('G total \n', G_tot)
      
      # print('============================================')
      # print('Converting Gao full back into MO basis')
      G_mo = ao2mo(G_tot[0], m.C_mo, full=False) # G in MO basis #! [0] not batched yet
      # print('G_mo \n', G_mo)
      # multiply by MO differencies
      # ii=0
      # for p in range(m.nocc):
      #  # print('p', p)
      #   for h in range(m.nocc, m.norb):
      #          # print('h', h)
      #          # print('i', i)
      #           f = m.e_mo[0][h] - m.e_mo[0][p]
      #           G_mo[ii] = G_mo[ii] + f * eta_orig[ii]
      #           G_mo[ii+N_cis] = -G_mo[ii + N_cis] + f * eta_orig[ii+N_cis]
      #           ii += 1
                
      vexp[:,i] = G_mo
      # print('G_mo \n', G_mo)

#-------------loop end----------------
  #  print('vexp.shape', vexp.shape)
  #  print('vexp\n', vexp)
  #  return vexp[:, :n_V_start]
   return vexp
 #  if CIS == True:
 # if CIS == False: # RPA

  # ! PART with orbtial energies seems to be guess vector construction G x V
  # TODO: vectorize so V.T x G x V could be constructed easily

In [1062]:
davidson_qm()

vexp1.shape torch.Size([8, 10])
vexp1
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
eta_orig.shape torch.Size([16])
eta_orig
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
G_mo.shape torch.Size([16])
G_mo
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<IndexPutBackward0>)
eta_orig.shape torch.Size([16])
eta_orig
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
G_mo.shape torch.Size([16])
G_mo
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<IndexPutBackward0>)
eta_orig.shape torch.Size([16])
eta_orig
 tensor([0., 0., 

In [1021]:
eval, evec = davidson(n_V_start = 4, 
         n_V_max = 10, 
         n_iter = 10, 
         keep_n = 3, 
         tol = 1e-3,)

print('eval\n', eval)
print('evec\n', evec)

molecules Molecule(
  (const): Constants()
  (parser): Parser()
)
vexp1.shape torch.Size([16, 4])
vexp1
 tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]])
ITERATION:  0
SUBSPACE SIZE V:  torch.Size([16, 4])
M.shape torch.Size([16, 16])
M
 tensor([[    -0.00000,      0.00000,     -0.66552,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,
              0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
        [     0.00000,      0.00000,      0.00000,     -0.06206,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,
  

RuntimeError: expected scalar type Double but found ComplexDouble

In [1031]:
def davidson(n_V, n_V_max, n_iter, keep_n, tol):
    
    # nd = 10 # dimension of Krylov subspace 
    # icount=0
    # nd1_old=0
    # n=0
    # iloop=0
    # itarget = 4
    # print('molecules', m)
    
    rrwork_sorted, indices_rpa = torch.sort(rrwork[:N_rpa], descending=False)
    vexp1 = torch.zeros((N_rpa, N_rpa), device=device)
    row_idx = torch.arange(0, int(N_rpa), device=device)
    col_idx = indices_rpa
    vexp1[row_idx, col_idx] = 1.0 
    
    vexp1 = vexp1[:, :n_V_start]
    #vexp1[:,itarget:] = 0.0 # return all columns beyond itarget to 0
    V = vexp1
    # vexp1
    
    
    print('vexp1.shape', vexp1.shape)
    print('vexp1\n', vexp1)
    
    # vexp1 is X in RPA?

   # V = torch.zeros((N_rpa, n_V_start), device=device)
    #V[:vexp1.shape[0], :vexp1.shape[1]] = vexp1[:, :n_V_start]
    
    # V = torch.zeros(16, n_V_start)

    # # Creating subpscace matrix with a diagonal guess based on original matrix
    # for i in range(n_V_start):
    #     # V[i, i] = diag[i] # koopmans guess??? not sure
    #     V[i, i] = 1
    # V = torch.linalg.qr(V)[0]


    iteration = 0
    term = False
    while iteration < n_iter and not term: # big loop
        
        print('=================================')
        print('ITERATION: ', iteration)
        print('SUBSPACE SIZE V: ', V.shape)
        print('=================================', flush=True)
        iteration += 1
#        V = torch.linalg.qr(V)[0]    # most likely, do not need. create_subspace_matrix already orthogonalizes; 
                                      # later on done before adding to matrix
        ### project original matrix in a subspace matrix
        M = davidson_qm(n_V)
        
        ray1 = torch.zeros((n_V, n_V), device=device)
        for i in range(n_V):
            for j in range(n_V):
                ray1[i,j] = torch.dot(vexp1[:,i], vexp[:N_cis,j])
        
        
        print('M.shape', M.shape)
        print('M\n', M)
        diag = torch.diag(M)
        sigma = M.T @ V   # or H|V>
        # # ic(sigma, sigma.shape)
       # M_V = V.T @ sigma # <V|H|V>
        # ic(M_V)
        
       # e_val, e_vec = torch.linalg.eig(M_V, UPLO='U') # find eigenvalues and eigenvectors #! EIGH or EIG?
        e_val, e_vec = torch.linalg.eigh(sigma)
        e_val = e_val.double() # complex --> double
        e_vec = e_vec.double()
        ic('eigenvalues BEFORE keep_n\n', e_val.shape)
        e_val_n = e_val[:keep_n] # keep only the lowest keep_n eigenvalues; full are still stored as e_val
        e_vec_n = e_vec[:, :keep_n]

        resids = torch.zeros(V.shape[0], len(e_val_n))
        ic(resids.shape)
        for j in range(len(e_val_n)): # calc residuals; probably ineffcienet - all calcd in loop, then norms calc'd
            r = sigma @ e_vec_n[:,j] - (e_val_n[j] * (V @ e_vec_n[:,j]))
            resids[:,j] = (sigma @ e_vec_n[:,j]) - (e_val_n[j] * (V @ e_vec_n[:,j])) # what if it should be just e_vec_n[:,j]?

        resids_norms = torch.tensor([resids[:,x].norm() for x in range(resids.shape[1])])
        ic(resids_norms, resids_norms.shape)
        if torch.any(resids_norms > tol): # check if any residual norm is larger than tol
#            print('at least one residual is too big', flush=True)
            mask = resids_norms >= tol
            large_res = resids[:,mask] # residuals larger than tol
 #           print('large residual norms ', resids_norms[mask])
            ic(large_res.shape)
            cor_e_val = e_val_n[mask] # corresponding eigenvalues !!! check if matches
            ic('V shape before adding residuals', V.shape)
            if large_res.shape[1] <= n_V_max - V.shape[1]: # startin adding residulas only if there is enough space
                                                           # otherwise when collapsing space, shapes will mismatch
                for j in range(large_res.shape[1]): # add new vectors to subspace matrix

                    if V.shape[1] < n_V_max: #keep_n is NEW HERE, CHECK 

                        denom = (diag[j] - cor_e_val[j]) 
                        s = large_res[:,j]/denom # conditioned residuals > tol
                        s = s/s.norm() # normalize
                        # !!! PROBABLY HIGHLY INEFFICIENT !!! 
                        # torch QR does not help! Returns square matrix only, we have m, m+1

                        V_new = torch.cat((V, torch.unsqueeze(s, dim=1)), dim=1) # add new vector as column to V
                        V_ort = orthogonalize(V_new) # orthogonalize V space with new vector, again should be translated to PyTorch
                        s_ort = V_ort[:, -1] # orthogonalized against current subspace

                        ratio = s_ort.norm()/resids[:,j].norm() # ratio of old and otogonalized norms
                        if ratio >= tol:
                            V = torch.column_stack((V, s_ort))
                        else:
                            pass

            else:
                print('!!!! MAX subspace reached !!!!', flush=True)
                ic('current shape of eigenvector matrix', e_vec.shape) 
                V = V @ e_vec[:,:n_V_start] # collapse subspace matrix to n_V_start vectors based on current eigenvectors
                ic(V.shape, e_vec[:,:n_V_start].shape)
                ic('forming new subspace from current eigenvectors')
                ic('subspace after max reached', V.shape)
                continue

        else:
            term = True
            print('============================')
            print('all residuals are below tolerance')
            print('DAVIDSON ALGORITHM CONVERGED')
            print('============================', flush=True)
            # print('eigenvalues: ', e_val, flush=True)
            # print('eigenvectors: ', e_vec, flush=True)
            return e_val, e_vec

    # runs after big loop if did not converge
    print('============================')
    print('!!! DAVIDSON ALGORITHM DID NOT CONVERGE !!!')
    print('============================', flush=True)
    # print('eigenvalues: ', e_val, flush=True)
    # print('eigenvectors: ', e_vec, flush=True)
    return e_val, e_vec

In [1032]:
eval, evec = davidson(n_V_start = 3, 
         n_V_max = 12, 
         n_iter = 10, 
         keep_n = 3, 
         tol = 1e-6,)

print('eval\n', eval)
print('evec\n', evec)

molecules Molecule(
  (const): Constants()
  (parser): Parser()
)
vexp1.shape torch.Size([16, 3])
vexp1
 tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.]])
ITERATION:  0
SUBSPACE SIZE V:  torch.Size([16, 3])
M.shape torch.Size([16, 16])
M
 tensor([[    -0.00000,      0.00000,     -0.66552,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,
              0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000],
        [     0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,      0.00000,
              0.00000,      0.00000,      0.00000,      0.00000,  

In [1030]:
def orthogonalize(U, eps=1e-15):
    """
    Orthogonalizes the matrix U (d x n) using Gram-Schmidt Orthogonalization.
    If the columns of U are linearly dependent with rank(U) = r, the last n-r columns 
    will be 0.
    
    Args:
        U (numpy.array): A d x n matrix with columns that need to be orthogonalized.
        eps (float): Threshold value below which numbers are regarded as 0 (default=1e-15).
    
    Returns:
        (numpy.array): A d x n orthogonal matrix. If the input matrix U's cols were
            not linearly independent, then the last n-r cols are zeros.
    
    Examples:
    ```python
    >>> import numpy as np
    >>> import gram_schmidt as gs
    >>> gs.orthogonalize(np.array([[10., 3.], [7., 8.]]))
    array([[ 0.81923192, -0.57346234],
       [ 0.57346234,  0.81923192]])
    >>> gs.orthogonalize(np.array([[10., 3., 4., 8.], [7., 8., 6., 1.]]))
    array([[ 0.81923192 -0.57346234  0.          0.        ]
       [ 0.57346234  0.81923192  0.          0.        ]])
    ```
    """
    
    n = len(U[0])
    # numpy can readily reference rows using indices, but referencing full rows is a little
    # dirty. So, work with transpose(U)
    V = U.T
    for i in range(n):
        prev_basis = V[0:i]     # orthonormal basis before V[i]
        coeff_vec = prev_basis @ V[i].T  # each entry is np.dot(V[j], V[i]) for all j < i
        # subtract projections of V[i] onto already determined basis V[0:i]
        V[i] -= (coeff_vec @ prev_basis).T
        if torch.norm(V[i]) < eps:
            V[i][V[i] < eps] = 0.   # set the small entries to 0
        else:
            V[i] /= torch.norm(V[i])
    return V.T

In [196]:
gsp = torch.tensor([10.6212,  0.0000,  0.0000])

In [ ]:
iter()

In [726]:
[x for x in range(3)]

[0, 1, 2]

In [205]:
for ii in range(5):
    H[ii] += 1
    print(H[ii])

tensor(2)
tensor(3)
tensor(4)
tensor(5)
tensor(6)


In [200]:
H

tensor([1, 2, 3, 4, 5])

In [196]:
H[2](0)

TypeError: 'Tensor' object is not callable

In [175]:
orb_loc = torch.zeros(m.nHeavy + m.nHydro, device=device)

In [172]:
orb_loc

tensor([0., 0., 0.])

In [168]:
# for i in orb_loc:
#     if m.Z[i] != 1:
#         loc = 
#     orb_loc[i] = i if

tensor([0., 0., 0., 0., 0., 0.])

In [153]:
m.Z

tensor([8, 1, 1])

In [113]:
torch.set_printoptions(precision=5, linewidth=200, sci_mode=False)

In [111]:
m.w

tensor([[[10.66662,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
         [ 0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
         [10.20110,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
         [ 0.89358,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
         [ 0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
         [10.69294,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
         [ 0.70142,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
         [ 0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
         [ 0.38607,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000],
 

In [112]:
m.w.shape

torch.Size([3, 10, 10])

In [35]:
rrwork[0]

tensor(40.6148)

In [36]:
#-------------Davidson ---------------------

# TODO: restart

# eta
# vexp1.shape = (8, 10) #! CIS guess vector
# vexp                  #! RPA guess vector
# eta is copy of vexp1
# TODO: figure vexp1
# probably orthogonalization could be ommited, if I can print it before and after

# TODO: figure j0

In [28]:
nd  = 10 # dimension of Krylov subspace


In [156]:
vexp1 = torch.zeros((N_cis, nd), device=device)

In [119]:
itarget = 4

In [155]:
row_idx = torch.arange(0, int(N_cis), device=device)
col_idx = indices

In [150]:
row_idx

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [151]:
col_idx

tensor([6, 7, 4, 5])

In [157]:
vexp1[row_idx, col_idx] = 1.0 

In [158]:
vexp1[:,itarget:] = 0.0 # return all columns beyond itarget to 0

In [159]:
vexp1 

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [147]:
f[:, :]

IndexError: too many indices for tensor of dimension 1

In [114]:
vexp1

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [201]:
ao2mo(m.C_mo[0], m.C_mo[0])

tensor([[-8.7887e-01, -1.2760e-15,  3.4020e-01,  7.2826e-16, -3.3445e-01,
         -1.0343e-15],
        [-3.7279e-16, -8.3358e-16, -6.5134e-16,  1.0000e+00, -1.4109e-16,
          7.8232e-17],
        [-2.9697e-16, -7.7088e-01, -3.4370e-15,  3.3837e-16, -1.2834e-15,
          6.3698e-01],
        [-1.0818e-01,  3.4058e-15, -8.2490e-01, -6.6271e-16, -5.5482e-01,
         -9.5833e-16],
        [-3.2855e-01, -4.5041e-01, -3.1921e-01, -2.0714e-16,  5.3866e-01,
         -5.4510e-01],
        [-3.2855e-01,  4.5041e-01, -3.1921e-01, -4.1659e-16,  5.3866e-01,
          5.4510e-01]])

In [738]:
def davidson():
    
   icount=0
   nd1_old=0
 #  m=0
   n=0
   iloop=0
   itarget = 4
   # goto 70 - restart
   
   nd  = 10 # dimension of Krylov subspace; usually 5x of excitations to be computed
   
   vexp1 = torch.zeros((N_cis, nd), device=device)
   row_idx = torch.arange(0, int(N_cis), device=device)
   col_idx = indices
   
   vexp1[row_idx, col_idx] = 1.0 
   vexp1[:,itarget:] = 0.0 # return all columns beyond itarget to 0
   
   print('vexp1.shape', vexp1.shape)
   print('vexp1\n', vexp1)
   
   import torch.nn.functional as F
   M = 4  # Size of the desired block
   pad_size = 6  # Number of rows/columns to pad
   
   # vexp1 is X in RPA?

   # vexp = F.pad(vexp1, (0, 0, N_cis, 0))
   
   # vexp = vexp1
   # print('vexp.shape', vexp.shape)
   # print('vexp\n', vexp)
   
   # ORTHONORMALIZATION
   
   
   
   # def Lxi(vexp):
      
   #    vexp = ao2mo(vexp, m.C_mo[0]) # vexp to MO basis
      
      
   # def Vxi(vexp):
      
   for i in range(4):  # i=nd1_old+1,nd1  
      
      # Lxi testing
      eta = torch.zeros((N_rpa), device=device)
      eta[:vexp1[:,i].shape[0]] = vexp1[:,i] # eta is stored as |X|
      print(eta)                             #                  |Y|
      
# --------------------------mo2site--------------------------------
      
      # dgemm_x mimics 3 dgemm calls in lioville.F90 NEXMD
      # probably noy efficient in torch

      # print(eta.size())
      # print(eta.shape)
      eta1 = eta[:N_cis]
      eta1 = eta1.view(-1, m.nvirt[0]) # 1d -> 2d
      print(eta1.shape)
      print('eta1', eta1)
      print('==============')
      
      eta_mo = torch.zeros((m.norb, m.norb), device=device)

      dgemm1 = eta1 @ m.C_mo[0][:, m.nocc:m.norb].T # operations on |X| ?

      # print('dgemm1.shape', dgemm1.shape)
      # print(dgemm1)
      
      eta_mo[:m.nocc] = dgemm1
      # print('eta_mo', eta_mo.shape)
      # print(eta_mo)
      
      
      eta2 = eta[N_cis:]                            # operations on |Y| ?
      eta2 = eta2.view(-1, m.nvirt[0]) # 1d -> 2d
   
      dgemm2 = eta2.T @ m.C_mo[0][:, :m.nocc].T
      
      # print('dgemm2.shape', dgemm2.shape)
      # print(dgemm2)
      
      eta_mo[m.nocc:] = dgemm2
      # print('eta_mo', eta_mo.shape)
      # print(eta_mo)
      
      dgemm3 = m.C_mo[0] @ eta_mo
      eta_mo = dgemm3 
      
      # print('dgemm3.shape', dgemm3.shape)
      # print(dgemm3)
      
# --------------------end mo2site--------------------------------
      
#--------------------------- Vxi ---------------------

      xi = eta_mo
      
      # xis size is Nb*(Nb+1)/2; 21 for H2O - store diagonal
      
      # symmertic part
      
      # dest_matrix[range(len(dest_matrix)), range(len(dest_matrix))] = source_vector
      # xi_sym = 0.5 * (xi + xi.T)
      # print('xi_sym.shape', xi_sym.shape)
      
      # xi_asym = 0.5 * (xi - xi.T)
      # print('xi_asym.shape', xi_asym.shape)
      
      #----upper matrix traingle: 21 elements for H2O-----
      
      xis = torch.zeros((m.norb*(m.norb+1)//2), device=device) # TODO: do via sym/antisym matrix decomposition
      l = 0
      for i in range(m.norb):
          #  print('i', i)
            for j in range(i+1):
              #    print('j', j)
                  
   #               print('l', l)
                  xis[l] = 0.5 * (xi[i,j] + xi[j,i])
                  l += 1
                  
      print('xis.shape', xis.shape)
      print('xis\n', xis)
      # -------- triangular -> 2d -------------------  
      
      # tri_idx = torch.triu_indices(int(6), int(6)) # indices of lower traingle [row][col]
      # xis_tri = torch.zeros((6, 6), device=device) 
      # xis_tri[tri_idx[0], tri_idx[1]] = xis # fill lower traingle with xis
      # xis_full = xis_tri + xis_tri.T - torch.diag(torch.diag(xis_tri))
      # print('xis_full\n', xis_full)
      
      xis_full = torch.zeros((6, 6), device='cpu') 

      l = 0
      for i in range(0, 6): # TODO" vectorize
          for j in range(i):
              l += 1
              xis_full[i,j] += xis[l-1]
              xis_full[j,i] += xis[l-1]
          l += 1 
          xis_full[i,i] += xis[l-1]
      
      
      
      # ------------------Vxi pack----------------------
      # xis is not changed in Vxi_pack call
      # eta = fock(nmol     = molecules.nmol[0],
      #            molsize  = molecules.molsize[0],
      #            P0       = xis,
      #            M        = torch.zeros((nmol*molsize**2,4,4))
      #            maskd    = m.maskd,
      mask     = m.mask, 
      idxi     = m.idxi,
      idxj     = m.idxj,
      w        = m.w,
                 

      
      # taken from fock in PYSEQM
      
      nmol = m.nmol
      molsize = m.molsize
      maskd = m.maskd
      idxi = m.idxi
      idxj = m.idxj
      w = m.w
      # TODO extract gxx programmatically
      gss = torch.tensor([15.7558, 14.7942, 14.7942])
      gpp = torch.tensor([13.6540,  0.0000,  0.0000])
      gsp = torch.tensor([10.6212,  0.0000,  0.0000])
      gp2 = torch.tensor([12.4061,  0.0000,  0.0000])
      hsp = torch.tensor([0.5939, 0.0000, 0.0000])

      F = torch.zeros((nmol*molsize**2,4,4), device=device) # 0 Fock matrix to fill
      # # TODO: feed params programmatically
      P0 = unpack(xis_full, 1, 2, 12) # 
      P0 = torch.unsqueeze(P0, 0) # add dimension
      
      print('P0.shape', P0.shape)
      print('P0\n', P0)
      #---------------fill diagonal 1c-2e -------------------
      P = P0.reshape((nmol,molsize,4,molsize,4)) \
          .transpose(2,3).reshape(nmol*molsize*molsize,4,4)
          
      print('P.shape', P.shape)
      print('P\n', P)
      
      Pptot = P[...,1,1]+P[...,2,2]+P[...,3,3]
      ## http://openmopac.net/manual/1c2e.html
    #  (s,s)
      TMP = torch.zeros_like(F)
      TMP[maskd,0,0] = 0.5*P[maskd,0,0]*gss + Pptot[maskd]*(gsp-0.5*hsp)
      for i in range(1,4):
          #(p,p)
          TMP[maskd,i,i] = P[maskd,0,0]*(gsp-0.5*hsp) + 0.5*P[maskd,i,i]*gpp \
                          + (Pptot[maskd] - P[maskd,i,i]) * (1.25*gp2-0.25*gpp)
          #(s,p) = (p,s) upper triangle
          TMP[maskd,0,i] = P[maskd,0,i]*(1.5*hsp - 0.5*gsp)
      #(p,p*)
      for i,j in [(1,2),(1,3),(2,3)]:
          TMP[maskd,i,j] = P[maskd,i,j]* (0.75*gpp - 1.25*gp2)

      F.add_(TMP)
      
           
      #-----------------fill 2c-2e integrals----------------
      weight = torch.tensor([1.0,
                        2.0, 1.0,
                        2.0, 2.0, 1.0,
                        2.0, 2.0, 2.0, 1.0],dtype=dtype, device=device).reshape((-1,10))
      
      PA = (P[maskd[idxi]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,10,1))
      PB = (P[maskd[idxj]][...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)]*weight).reshape((-1,1,10))
      suma = torch.sum(PA*w,dim=1)
      sumb = torch.sum(PB*w,dim=2)
      sumA = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      sumB = torch.zeros_like(sumA)
      
      sumA[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = suma
      sumB[...,(0,0,1,0,1,2,0,1,2,3),(0,1,1,2,2,2,3,3,3,3)] = sumb
      F.index_add_(0,maskd[idxi],sumB)
      #\sum_A
      F.index_add_(0,maskd[idxj],sumA)
      
      sum = torch.zeros(w.shape[0],4,4,dtype=dtype, device=device)
      # (ss ), (px s), (px px), (py s), (py px), (py py), (pz s), (pz px), (pz py), (pz pz)
      #   0,     1         2       3       4         5       6      7         8        9

      ind = torch.tensor([[0,1,3,6],
                          [1,2,4,7],
                          [3,4,5,8],
                          [6,7,8,9]],dtype=torch.int64, device=device)
      
      Pp = -0.5*P[mask]
      for i in range(4):
        for j in range(4):
            #\sum_{nu \in A} \sum_{sigma \in B} P_{nu, sigma} * (mu nu, lambda, sigma)
            sum[...,i,j] = torch.sum(Pp*w[...,ind[i],:][...,:,ind[j]],dim=(1,2))
    #  print('mask', mask[0])    #! DIFFERS FROM PYSEQM, PROBABLY packing
      F.index_add_(0,mask[0],sum)

      F0 = F.reshape(nmol,molsize,molsize,4,4) \
             .transpose(2,3) \
             .reshape(nmol, 4*molsize, 4*molsize)
    #
      F0.add_(F0.triu(1).transpose(1,2))     
      
      F0 = 2 * F0 #! BE CAREFUL
      print('F0.shape', F0.shape)
      print(F0) 
      break


In [497]:
davidson()

vexp1.shape torch.Size([8, 10])
vexp1
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
torch.Size([4, 2])
eta1 tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.]])
xis.shape torch.Size([21])
xis
 tensor([    -0.00000,     -0.16722,     -0.00000,      0.00000,     -0.00000,      0.00000,      0.00000,     -0.27741,      0.00000,      0.00000,      0.00000,      0.26933,     -0.00000,
            -0.00000,     -0.00000,      0.00000,      0.26933,     -0.00000,     -0.00000,     -0.00000,     -0.00000])
P0.shape torch.Size([1, 12, 12])
P0
 tensor([[[

In [409]:
list(m.parameters())

[Parameter containing:
 tensor([0., 1., 0., 1., 2., 3., 4., 5., 6., 7., 0., 1., 2., 3., 4., 5., 6., 7., 0.]),
 Parameter containing:
 tensor([0., 1., 0., 2., 2., 2., 2., 2., 2., 2., 0., 3., 3., 3., 3., 3., 3., 3., 0.]),
 Parameter containing:
 tensor([0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 3, 3, 3, 3, 3, 3, 3, 0]),
 Parameter containing:
 tensor([0., 1., 0., 1., 2., 2., 2., 2., 2., 2., 0., 1., 2., 2., 2., 2., 2., 2., 0.]),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 1., 2., 3., 4., 5., 6., 0., 0., 1., 2., 3., 4., 5., 6.]),
 Parameter containing:
 tensor([0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.]),
 Parameter containing:
 tensor([ 0.,  0.,  0.,  0.,  0.,  2.,  4.,  6.,  8., 10.,  0.,  0.,  0.,  2.,  4.,  6.,  8., 10.,  0.]),
 Parameter containing:
 tensor([ 0.,  0.,  0.,  0.,  0., -1., -2., -3., -4., -5.,  0.,  0.,  0., -1., -2., -3., -4., -5.,  0.]),
 Parameter containing:
 tensor([ 0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  0.00000,  1.50000,

In [410]:
list(m.named_parameters())

[('const.tore',
  Parameter containing:
  tensor([0., 1., 0., 1., 2., 3., 4., 5., 6., 7., 0., 1., 2., 3., 4., 5., 6., 7., 0.])),
 ('const.qn',
  Parameter containing:
  tensor([0., 1., 0., 2., 2., 2., 2., 2., 2., 2., 0., 3., 3., 3., 3., 3., 3., 3., 0.])),
 ('const.qn_int',
  Parameter containing:
  tensor([0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 3, 3, 3, 3, 3, 3, 3, 0])),
 ('const.ussc',
  Parameter containing:
  tensor([0., 1., 0., 1., 2., 2., 2., 2., 2., 2., 0., 1., 2., 2., 2., 2., 2., 2., 0.])),
 ('const.uppc',
  Parameter containing:
  tensor([0., 0., 0., 0., 0., 1., 2., 3., 4., 5., 6., 0., 0., 1., 2., 3., 4., 5., 6.])),
 ('const.gssc',
  Parameter containing:
  tensor([0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.])),
 ('const.gspc',
  Parameter containing:
  tensor([ 0.,  0.,  0.,  0.,  0.,  2.,  4.,  6.,  8., 10.,  0.,  0.,  0.,  2.,  4.,  6.,  8., 10.,  0.])),
 ('const.hspc',
  Parameter containing:
  tensor([ 0.,  0.,  0.,  0.,  0., -1., -2., -3., -4., -5

In [415]:
m.parameters

{'U_ss': tensor([-86.99300, -13.07332, -13.07332]),
 'U_pp': tensor([-71.87958,   0.00000,   0.00000]),
 'zeta_s': tensor([3.79654, 0.96781, 0.96781]),
 'zeta_p': tensor([2.38940, 0.00000, 0.00000]),
 'beta_s': tensor([-45.20265,  -5.62651,  -5.62651]),
 'beta_p': tensor([-24.75251,   0.00000,   0.00000]),
 'g_ss': tensor([15.75576, 14.79421, 14.79421]),
 'g_sp': tensor([10.62116,  0.00000,  0.00000]),
 'g_pp': tensor([13.65402,  0.00000,  0.00000]),
 'g_p2': tensor([12.40610,  0.00000,  0.00000]),
 'h_sp': tensor([0.59388, 0.00000, 0.00000]),
 'alpha': tensor([3.21710, 3.35639, 3.35639]),
 'Gaussian1_K': tensor([-1.13113,  1.12875,  1.12875]),
 'Gaussian2_K': tensor([ 1.13789, -1.06033, -1.06033]),
 'Gaussian1_L': tensor([6.00248, 5.09628, 5.09628]),
 'Gaussian2_L': tensor([5.95051, 6.00379, 6.00379]),
 'Gaussian1_M': tensor([1.60731, 1.53747, 1.53747]),
 'Gaussian2_M': tensor([1.59840, 1.57019, 1.57019])}

In [476]:
def davidson_vec():
    
   icount=0
   nd1_old=0
 #  m=0
   n=0
   iloop=0
   itarget = 4
   print('molecules', m)
   # goto 70 - restart
   
   nd  = 10 # dimension of Krylov subspace; usually 5x of excitations to be computed
   
   vexp1 = torch.zeros((N_cis, nd), device=device)
   row_idx = torch.arange(0, int(N_cis), device=device)
   col_idx = indices
   
   vexp1[row_idx, col_idx] = 1.0 
   vexp1[:,itarget:] = 0.0 # return all columns beyond itarget to 0
   
   print('vexp1.shape', vexp1.shape)
   print('vexp1\n', vexp1)
   
   import torch.nn.functional as F
   M = 4  # Size of the desired block
   pad_size = 6  # Number of rows/columns to pad
   
   # vexp1 is X in RPA?

   # vexp = F.pad(vexp1, (0, 0, N_cis, 0))
   
   # vexp = vexp1
   # print('vexp.shape', vexp.shape)
   # print('vexp\n', vexp)
   
   # ORTHONORMALIZATION
   # parse params for G routines
   gss = m.parameters['g_ss']
   gsp = m.parameters['g_sp']
   gpp = m.parameters['g_pp']
   gp2 = m.parameters['g_p2']
   hsp = m.parameters['h_sp']
   
   mask  = m.mask
   maskd = m.maskd
   idxi  = m.idxi
   idxj  = m.idxj
   nmol  = m.nmol
   molsize = m.molsize
   w       = m.w
   nHeavy = m.nHeavy
   nHydro = m.nHydro
   
   
      
   for i in range(4):  # i=nd1_old+1,nd1  
      
      # Lxi testing
      eta = torch.zeros((N_rpa), device=device)
      eta[:vexp1[:,i].shape[0]] = vexp1[:,i] # eta is stored as |X|
      
      print('eta.shape', eta.shape)
      print(eta)
      eta_ao =  mo2ao(eta, m.C_mo[0], full=False)     # mo to ao basis (mo2site)
      # print('eta_ao.shape', eta_ao.shape)
      # print(eta_ao)

      eta_ao_sym, eta_ao_asym = decompose_to_sym_antisym(eta_ao) # decompose to sym and asym
    
      # Vxi - build 2e integrals in AO basis: G(guess density) in F = H_core + G
      # note density is split into sym and anisym matrices
      # sym is processed as padded 3d array in PYSEQM, see fock module 
      # antisym: 2c-2e works with modified PYSEQM routine; should be antisimmterized afterwards
      # antisym: 1c-2e (diagonal) are taken from NEXMD for now - ugly code with loops
      # TODO: vectorize 1c-2e part
      
      #------------------symmetric------------------------------
      G_sym   =  build_G_sym(eta_ao_sym,
                            gss, gsp, gpp, gp2, hsp,
                            mask, maskd, idxi, idxj, nmol, molsize,
                            w)
      # pack 2c-2e part to standard shape
      G_ao_sym = pack.pack(G_ao_sym, nHeavy, nHydro)
      
      #------------------end symmetric--------------------------
     
      # -----------antysim manual------------
      # TODO: adapt build_G for asym part
      l = 0
      
      # make triangular 1d array as in Vxi
      eta_anti = torch.zeros((m.norb*(m.norb+1)//2), device=device)
      
      for i in range(m.norb):
          #  print('i', i)
            for j in range(i+1):
              #    print('j', j)
                  
   #               print('l', l)
                  eta_anti[l] = 0.5 * (eta_ao[i,j] - eta_ao[j,i])
                  l += 1
                  
     # print('eta_anti.shape', eta_anti.shape)
     # print(eta_anti)
      
      eta_anti_2d = torch.zeros((6, 6), device='cpu') 

      # restore to 2d form to build G 2c-2e part
      # l = 0
      # for i in range(0, 6): # TODO" vectorize
      #     for j in range(0,i):
      #         l += 1
      #         eta_anti_2d[i,j] += eta_anti[l-1]
      #         eta_anti_2d[j,i] -= eta_anti[l-1]
      #     l += 1 

      # print('eta_anti_2d.shape', eta_anti_2d.shape)
      # print(eta_anti_2d)
        
      # 2c-2e part of G from antisymmetric guess
      G_ao_antisym_2c2e = build_G_2c2e(eta_ao_asym, m)
      #print('G_ao_antisym_2c2e shape before packing', G_ao_antisym_2c2e.shape)
      G_ao_antisym_2c2e = pack.pack(G_ao_antisym_2c2e, nHeavy, nHydro)
      
      # print('2c-2e antisym\n', G_ao_antisym_2c2e)
      
    #   #--------------fock skew for 1d-array of 1c-2e part of G----------------
      
    #   mask     = m.mask, 
    #   idxi     = m.idxi,
    #   idxj     = m.idxj,
    #   w        = m.w,
    #   nmol = m.nmol
    #   molsize = m.molsize
    #   maskd = m.maskd
    #   idxi = m.idxi
    #   idxj = m.idxj
    #   w = m.w
      
    #   gss = torch.tensor([15.7558, 14.7942, 14.7942])  # GSSII   7.87788  7.39710   7.39710   1/2 # not used
    #   gpp = torch.tensor([13.6540,  0.0000,  0.0000])  # GPPII   6.82700   1/2
    #   gsp = torch.tensor([10.6212,  0.0000,  0.0000])  # GSPII   10.6211    
    #   gp2 = torch.tensor([12.4061,  0.0000,  0.0000])  # GP2II   15.5076
    #   hsp = torch.tensor([0.5939, 0.0000, 0.0000])     # HSPII   0.29694   1/2
      
      
    #   print('maskd', maskd)
    #   F = torch.zeros((nmol*molsize**2,4,4), device=device) # 0 Fock matrix to fill
    #   # # TODO: feed params programmatically
    #   P0 = unpack(eta_ao_asym, 1, 2, 12) # 
    #   P0 = torch.unsqueeze(P0, 0) # add dimension
      
    #   # print('P0.shape', P0.shape)
    #   # print('P0\n', P0)
    #   #---------------fill diagonal 1c-2e -------------------
    #   P = P0.reshape((nmol,molsize,4,molsize,4)) \
    #       .transpose(2,3).reshape(nmol*molsize*molsize,4,4)
          
    #   # print('P.shape', P.shape)
    #   # print('P\n', P)
      
    #   Pptot = P[...,1,1]+P[...,2,2]+P[...,3,3]
      
    #   ## http://openmopac.net/manual/1c2e.html
    # #  (s,s)
    #   TMP = torch.zeros_like(F)
    #   TMP[maskd,0,0] =  0.5*Pptot[maskd]*(hsp-gsp)
    #   for i in range(1,4):
    #       #(p,p)
    #       TMP[maskd,i,i] = P[maskd,0,0]*(hsp-gsp) + 0.0*P[maskd,i,i]*gpp \
    #                       + (Pptot[maskd] - P[maskd,i,i]) * (0.25*gpp - 0.60*15.5076)
    #       #(s,p) = (p,s) upper triangle
    #       TMP[maskd,0,i] = P[maskd,0,i]*(hsp-gsp)
    #   #(p,p*)
    #   for i,j in [(1,2),(1,3),(2,3)]:
    #       TMP[maskd,i,j] = P[maskd,i,j]* (0.25*gpp - 0.60*15.5076) #gp2

    #   F.add_(TMP)

      
    #   F0 = F.reshape(nmol,molsize,molsize,4,4) \
    #          .transpose(2,3) \
    #          .reshape(nmol, 4*molsize, 4*molsize)
    # #
    #   F0.add_(F0.triu(1).transpose(1,2))     
      
    #   F0 = 2 * F0 #! BE CAREFUL
      
    #  # print('F 1c-2e part of antisym', F.shape)
    #   F0 = pack.pack(F0, nHeavy, nHydro)
    #   print('F sym part\n', G_ao_sym)
    #   print('F antisym 2c-2e part\n', G_ao_antisym_2c2e)
    #   print('F 1c-2e part of antisym\n', F0)
    #   print('F total\n',  G_ao_sym + (2*F0 + G_ao_antisym_2c2e))
    # ORIGINAL
    #   TMP = torch.zeros_like(F)
    #   TMP[maskd,0,0] = 0.5*P[maskd,0,0]*gss + Pptot[maskd]*(gsp-0.5*hsp)
    #   for i in range(1,4):
    #       #(p,p)
    #       TMP[maskd,i,i] = P[maskd,0,0]*(gsp-0.5*hsp) + 0.5*P[maskd,i,i]*gpp \
    #                       + (Pptot[maskd] - P[maskd,i,i]) * (1.25*gp2-0.25*gpp)
    #       #(s,p) = (p,s) upper triangle
    #       TMP[maskd,0,i] = P[maskd,0,i]*(1.5*hsp - 0.5*gsp)
    #   #(p,p*)
    #   for i,j in [(1,2),(1,3),(2,3)]:
    #       TMP[maskd,i,j] = P[maskd,i,j]* (0.75*gpp - 1.25*gp2)

    #   F.add_(TMP)
      
      
      pascal2 = [1, 3, 6, 10, 15, 21]  # -1 pascal triangle
      pascal2 = [x -1 for x in pascal2]
      pascal1 = [0, 1, 3, 6, 10, 15] # -1 for python indexing
      pascal1 = [x -1 for x in pascal1]
      orb_loc1 = [0,4,5] # O orbs
      orb_loc2 = [3,4,5]
      F = torch.zeros((m.norb*(m.norb+1)//2))
      # print('F.shape', F.shape)
      # print(F)
      gss = torch.tensor([15.7558, 14.7942, 14.7942])  # GSSII   7.87788  7.39710   7.39710   1/2 # not used
      gpp = torch.tensor([13.6540,  0.0000,  0.0000])  # GPPII   6.82700   1/2
      gsp = torch.tensor([10.6212,  0.0000,  0.0000])  # GSPII   10.6211    
      gp2 = torch.tensor([15.5076,  0.0000,  0.0000])  # GP2II   15.5076   #! take from fortran
      hsp = torch.tensor([0.5939, 0.0000, 0.0000])     # HSPII   0.29694   1/2
      
      print('==============================================')
      for ii in range(3): # n_atoms?
          # print('ii', ii)
          gsp_ii = gsp[ii]
          gpp_ii = gpp[ii]
          gp2_ii = gp2[ii]
          hsp_ii = hsp[ii]
          
          ia = orb_loc1[ii]
          # print('ia', ia)
          ib = orb_loc2[ii]
          
          iplus = ia+1
          ka = pascal2[ia]
          l = ka
          for j in range(iplus, ib+1):
            # print('j', j)
            # print('ia', ia)
            # print('ib', ib)
            # print('l', l)

            mm = l+ia+1
          #  print('UPPER mm', mm)
            l = l+j+1
            
           # print(type(mm)) 
            F[mm] = F[mm] + 0.5*eta_anti[mm] * (hsp_ii - gsp_ii)
          #  print('(hsp - gsp)', (hsp - gsp))
           # print('F[mm]', F[mm])
          iminus = ib-1
          
          for j in range(iplus, iminus+1):
            icc = j
            # print('icc', icc)
            for l in range(icc, ib):
              # print('l', l)
              mm = pascal1[l+1] + j+1
             # print('mm', mm)
             # print('(0.25*gpp_ii - 0.6*gp2_ii)', (0.25*gpp_ii - 0.6*gp2_ii) )
              F[mm] = F[mm]+ eta_anti[mm] * (0.25*gpp_ii - 1.25*0.6*gp2_ii) 
             # print('F[mm]', F[mm])
      F = F*2
    #  print('F antisym\n', F)
      
      # ---------restore asym 1c-2e part to 2d matrix
      G_ao_antisym_1c_2e_1d = F  # as in last lines of build_G
      
      # G_ao_antisym_1c_2e = torch.zeros((6, 6), device='cpu') # for 2d form
      
      G = G_ao_sym[0]
   #   print('G shape\n', G.shape)
      
      l = 0
      for i in range(0, 6): # TODO" vectorize
          for j in range(0,i):
              l += 1
              # G[i,j] += G_ao_antisym_2c2e[0][i,j]*2 + G_ao_antisym_1c_2e_1d[l-1]
              # G[j,i] -= G_ao_antisym_2c2e[0][j,i]*2 - G_ao_antisym_1c_2e_1d[l-1]
              G[i,j] += G_ao_antisym_1c_2e_1d[l-1]
              G[j,i] -= G_ao_antisym_1c_2e_1d[l-1]             
          l += 1 # skip diagonal
      
      
      
      # l = 0
      # for i in range(0, 6): # TODO" vectorize
      #      for j in range(0,i):
      #          l += 1
      #          G_ao_antisym_1c_2e[i,j] += G_ao_antisym_1c_2e_1d[l-1]
      #          G_ao_antisym_1c_2e[j,i] += G_ao_antisym_1c_2e_1d[l-1]
      #      l += 1 
      #      G_ao_antisym_1c_2e[i,j] += G_ao_antisym_1c_2e_1d[l-1]
          
   #   print('G 2c-2e antisym\n', G_ao_antisym_2c2e)
    #  print('G 2c-2e antisym.shape\n', G_ao_antisym_2c2e.shape)
      G_ao_antisym_2c2e =  G_ao_antisym_2c2e[0]
  #     print('G_sym\n', G_ao_sym)
      
  #  #   print('G 1c-2e antisym \n', G_ao_antisym_1c_2e)
      rows, cols = torch.tril_indices(G_ao_antisym_2c2e.shape[0], G_ao_antisym_2c2e.shape[1])

# Multiply the lower triangle elements by -1
      G_ao_antisym_2c2e[rows, cols] *= -1

# Add the diagonal elements back
      G_ao_antisym_2c2e[torch.eye(G_ao_antisym_2c2e.shape[0]).bool()] *= -1


      print('G total \n', G + G_ao_antisym_2c2e*2)
      break
      
      # TODO: fock1_skew: antisym density is processed differently
      

